In [1]:
from __future__ import print_function
import sys

# This is not required if you've installed pycparser into
# your site-packages/ with setup.py
#
sys.path.extend(['.', '..'])

from pycparser import c_parser, c_ast
from pycparser import parse_file

In [2]:
ast = parse_file('./pycparser-master/examples/c_files/funky.c', use_cpp=True,
        cpp_path='gcc',
        cpp_args=['-E', r'-Iutils/fake_libc_include']
        )

In [50]:
ast.show(attrnames=True, nodenames=True)

FileAST: 
  FuncDef <ext[0]>: 
    Decl <decl>: name=foo, quals=['const'], storage=[], funcspec=[]
      FuncDecl <type>: 
        ParamList <args>: 
          Typename <params[0]>: name=None, quals=[]
            TypeDecl <type>: declname=None, quals=[]
              IdentifierType <type>: names=['void']
        TypeDecl <type>: declname=foo, quals=['const']
          IdentifierType <type>: names=['char']
    Compound <body>: 
      Decl <block_items[0]>: name=c, quals=['const'], storage=[], funcspec=[]
        TypeDecl <type>: declname=c, quals=['const']
          IdentifierType <type>: names=['char']
        Constant <init>: type=char, value='t'
      Return <block_items[1]>: 
        ID <expr>: name=c
  FuncDef <ext[1]>: 
    Decl <decl>: name=maxout_in, quals=[], storage=[], funcspec=[]
      FuncDecl <type>: 
        ParamList <args>: 
          Decl <params[0]>: name=paste, quals=[], storage=[], funcspec=[]
            TypeDecl <type>: declname=paste, quals=[]
              Iden

In [88]:
operators = {}
operands = {}
# broj pokazivaca za pointerdecl. anulira se u identifiertype
num_pointers = 0
# ne zanima me ID ako je deklaracija, inace me zanima
is_declaration = 0

In [89]:
def traverse(node):
    global num_pointers
    global is_declaration
    #print(node)

    
    '''
        pycparser prepoznaje 46 tipova cvorova stabla AST
    
        Potrebno je obici cvorove stabla i obraditi ih na odgovarajuci nacin
    '''
    # child od FileAST moze da bude Decl|FuncDef|Typedef
    
    if isinstance (node, c_ast.ArrayDecl):
        print("ArrayDecl")
        # ArrayDecl[type*, dim*, dim_quals*]
        
        if node.type:
            traverse(node.type)
        if node.dim:
            traverse(node.dim)
        if node.dim_quals:
            traverse(node.dim_quals)
        
    elif isinstance (node, c_ast.ArrayRef):
        print("ArrayRef")
        # ArrayRef[name*, subscript*]
        
        if node.name:
            traverse(node.name)
        if node.subscript:
            traverse(node.subscript)
        
    elif isinstance (node, c_ast.Assignment):
        print("Assignment")
        # Assignment[op, lvalue*, rvalue*]
        
        if node.op in operators:
            operators[node.op] += 1
        else:
            operators[node.op] = 1
        
        if ';' in operators:
            operators[';'] += 1
        else:
            operators[';'] = 1
                        
        if node.lvalue:
            traverse(node.lvalue)
        if node.rvalue:
            traverse(node.rvalue)
                
        
    elif isinstance (node, c_ast.BinaryOp):
        print("Binary op")
        # BinaryOp[op, left*, right*]
        # a = 5
        # a = b = i + 5 - l/3
        
        if node.op in operators:
            operators[node.op] += 1
        else: 
            operators[node.op] = 1
            
        # moraju da postoje left i right, provera je suvisna
        if node.left:
            traverse(node.left)
        if node.right:
            traverse(node.right)
        
        
    elif isinstance (node, c_ast.Break):
        print("Break")
        # Break[]
        
        if "break" in operators:
            operators["break"] += 1
        else:
            operators["break"] = 1
        if ';' in operators:
            operators[';'] += 1
        else:
            operators[';'] = 1            
        
    elif isinstance (node, c_ast.Case):
        print("Case")
        # Case[expr*, stmts**]
        
        # da li se dodaje ? ['case' i ':']
        
        if node.expr:
            traverse(node.expr)
        if node.stmts:
            for stmt in node.stmts:
                traverse(stmt)
        
        
    elif isinstance (node, c_ast.Cast):
        print("Cast")
        # Cast[to_type*, expr*] -> (int) l
        if '(' in operators:
            operators['('] += 1
        else:
            operators['('] = 1
        if ')' in operators:
            operators[')'] += 1
        else:
            operators[')'] = 1
        
        if node.to_type:
            traverse(node.to_type)
        if node.expr:
            traverse(node.expr)
        
    elif isinstance (node, c_ast.Compound):
        print("Compound")
        # Compound[block_items**]
        # ->  {
        #       ..., ..., 
        #       ...
        #      }
        print('{')
        
        if '{' in operators:
            operators['{'] += 1
        else:
            operators['{'] = 1
        if '}' in operators:
            operators['}'] += 1
        else:
            operators['}'] = 1
        
        if node.block_items:
            for item in node.block_items:
                traverse(item)
        
        print('}')
        
    elif isinstance (node, c_ast.CompoundLiteral):
        print("Compound literal")
        # CompoundLiteral[type*, init*]
        
        if node.type:
            traverse(node.type)
        if node.init:
            traverse(node.init)
        
    elif isinstance (node, c_ast.Constant):
        print("Constant")
        # Const[type, value]
        # U ovom slucaju ne treba nista da se radi jer je konstanta karakter 'a' ili broj 5, ...
        return
    elif isinstance (node, c_ast.Continue):
        print("Continue")
        #Continue[]
        
        if "continue" in operators:
            operators["continue"] += 1
        else:
            operators["continue"] = 1
        if ';' in operators:
            operators[';'] += 1
        else:
            operators[';'] = 1  
        
    elif isinstance (node, c_ast.Decl):
        print("Decl")
        # Decl[name, quals, storage, funspace, type*, init*, bitsize*]
        
        # name parametar nije znacajan zato sto imena funkcija i promenljivih prilikom deklaracija
        # ne spadaju u operatore tj. operande
        
        is_declaration += 1
        
        # const, volatile
        quals = node.quals        
        if quals: # if true
            if (len(quals) == 2):
                #print(quals[0]+"   "+quals[1])
                if (quals[0] in operators):
                    operators[quals[0]] += 1
                else:
                    operators[quals[0]] = 1
                if (quals[1] in operators):
                    operators[quals[1]] += 1
                else:
                    operators[quals[1]] = 1
            else: #(node.quals() == 1):
                #print(quals[0])
                if (quals[0] in operators):
                    operators[quals[0]] += 1
                else:
                    operators[quals[0]] = 1
                
        # extern, register
        storage = node.storage        
        if storage: # if true
            if (len(storage) == 2):
                #print(storage[0]+"   "+storage[1])
                if (storage[0] in operators):
                    operators[storage[0]] += 1
                else:
                    operators[storage[0]] = 1
                if (storage[1] in operators):
                    operators[storage[1]] += 1
                else:
                    operators[storage[1]] = 1
            else: #(len(storage) == 1):
                #print(storage[0])
                if (storage[0] in operators):
                    operators[storage[0]] += 1
                else:
                    operators[storage[0]] = 1   
        
        # funspec
        funcspec = node.funcspec
        if funcspec:
            print(funcspec[0])
            if (funcspec[0] in operators):
                operators[funcspec[0]] += 1
            else:
                operators[funcspec[0]] = 1  
        
        if node.type:
            traverse(node.type)
        if node.init:
            traverse(node.init)
        if node.bitsize:
            traverse(node.bitsize)
            
        if ';' in operators:
            operators[';'] += 1
        else:
            operators[';'] = 1
        
        is_declaration -= 1
                    
    elif isinstance (node, c_ast.DeclList):
        print("decl list")
        # DeclList[decls**]
        
        if node.decls:
            for decl in node.decls:
                traverse(decl)
        
    elif isinstance (node, c_ast.Default):
        print("default")
        # Default[stmts**]
        
        id node.stmts:
            for stmt in node.stmts:
                traverse(stmt)
        
    elif isinstance (node, c_ast.DoWhile):
        print("do while")
        # DoWhile[cond*, stmt*]
        '''
        do {
           statement(s);
        } while( condition );
        '''
        
        if "do while" in operators:
            operators["do while"] += 1
        else:
            operators["do while"] = 1
        if ';' in operators:
            operators[';'] += 1
        else:
            operators[';'] = 1 
        if '{' in operators:
            operators['{'] += 1
        else:
            operators['{'] = 1
        if '}' in operators:
            operators['}'] += 1
        else:
            operators['}'] = 1  
        
        if node.cond:
            traverse(node.cond)
        if node.stmt:
            traverse(node.stmt)        
        
    elif isinstance (node, c_ast.EllipsisParam):
        print("ellipsis param")
    elif isinstance (node, c_ast.EmptyStatement):
        print("empty statement")
    elif isinstance (node, c_ast.Enum):
        print("enum")
    elif isinstance (node, c_ast.Enumerator):
        print("Enumerator")
    elif isinstance (node, c_ast.EnumeratorList):
        print("Enumerator list")
    elif isinstance (node, c_ast.ExprList):
        print("Expr list")
    elif isinstance (node, c_ast.For):
        print("For")
    elif isinstance (node, c_ast.FuncCall):
        print("Func call")
    elif isinstance (node, c_ast.FuncDecl):
        print("Func decl") 
        # FuncDecl[args*, type*]
        if node.args:
            traverse(node.args)
        if node.type:
            traverse(node.type)
        
    elif isinstance (node, c_ast.FuncDef):
        print("FuncDef")
        # FuncDef [decl*, param_decls**, body*]
        node.decl.show(attrnames=True, nodenames=True)
        
        # oduzimamo ; jer ne postoji, a dodaje se u Decl koji se nalazi u okviru FuncDef
        if ';' in operators:
            operators[';'] -= 1
        else:
            operators[';'] = -1
        
        if node.decl:
            traverse(node.decl)
        if node.param_decls:
            for pd in node.param_decls:
                if pd:
                    traverse(pd)
        if node.body:
            traverse(node.body)
        
        # node.show(attrnames=True, nodenames=True)
        
        print('-----------------')
    elif isinstance (node, c_ast.Goto):
        print("goto")
    elif isinstance (node, c_ast.ID):
        print("ID")
        # ID[name]
        
        if not (is_declaration):
            if node.name in operands:
                operands[node.name] += 1
            else:
                operands[node.name] = 1
        
        return
        
    elif isinstance (node, c_ast.IdentifierType):
        print("Identifier Type")
        # IdentifierType[names] [long long int]
        
        id_type = ""
        if node.names:
            for name in node.names:
                id_type += name
                id_type += ' '
                
        # dodajem * koliko ima num_pointers
        for i in range(0,num_pointers):
            id_type += '*'
                
        if id_type in operands:
            operands[id_type] += 1
        else: 
            operands[id_type] = 1
            
        num_pointers = 0
        
    elif isinstance (node, c_ast.If):
        print("if")
    elif isinstance (node, c_ast.InitList):
        print("init list")
    elif isinstance (node, c_ast.Label):
        print("label")
    elif isinstance (node, c_ast.NamedInitializer):
        print("Named Initializer")
    elif isinstance (node, c_ast.ParamList):
        print("Param list")
        # ParamList[params**]
        if node.params:
            for param in node.params:
                print(param)
                if param:
                    traverse(param)
        '''
        ParamList <args>: 
          Typename <params[0]>: name=None, quals=[]
            TypeDecl <type>: declname=None, quals=[]
              IdentifierType <type>: names=['void']
        '''
    elif isinstance (node, c_ast.Pragma):
        print("Pragma")
    elif isinstance (node, c_ast.PtrDecl):
        print("PtrDecl")
        # PtrDecl[quals, type*]
        
        # const, volatile
        quals = node.quals        
        if quals: # if true
            if (len(quals) == 2):
                #print(quals[0]+"   "+quals[1])
                if (quals[0] in operators):
                    operators[quals[0]] += 1
                else:
                    operators[quals[0]] = 1
                if (quals[1] in operators):
                    operators[quals[1]] += 1
                else:
                    operators[quals[1]] = 1
            else: #(node.quals() == 1):
                #print(quals[0])
                if (quals[0] in operators):
                    operators[quals[0]] += 1
                else:
                    operators[quals[0]] = 1
                    
        num_pointers += 1
        
        if node.type:
            traverse(node.type)
        
    elif isinstance (node, c_ast.Return):
        print("Return")
        # Return[expr*]
        if "return" in operators:
            operators["return"] += 1
        else:
            operators["return"] = 1
            
        if ';' in operators:
            operators[';'] += 1
        else:
            operators[';'] = 1
        
        if node.expr:
            traverse(node.expr)
        
    elif isinstance (node, c_ast.Struct):
        print("struct")                  
    elif isinstance (node, c_ast.StructRef):
        print("Struct ref")
    elif isinstance (node, c_ast.Switch):
        print("switch")
    elif isinstance (node, c_ast.TernaryOp):
        print("ternary operator")
    elif isinstance (node, c_ast.TypeDecl):
        print("Type Decl")
        # TypeDecl[declname, quals, type*]
        
        # node.declname nije bitan jer je u pitanju deklaracija
        
        '''
        # izgleda da nema potrebe da se i ovde nalazi jer se nalazi u Decl
        
        # const, volatile
        quals = node.quals        
        if quals: # if true
            if (len(quals) == 2):
                print(quals[0]+"   "+quals[1])
                if (quals[0] in operators):
                    operators[quals[0]] += 1
                else:
                    operators[quals[0]] = 1
                if (quals[1] in operators):
                    operators[quals[1]] += 1
                else:
                    operators[quals[1]] = 1
            else: #(node.quals() == 1):
                #print("******************  "+node.quals[0])
                if (quals[0] in operators):
                    operators[quals[0]] += 1
                else:
                    operators[quals[0]] = 1
        '''
        if node.type:
            traverse(node.type)

    elif isinstance (node, c_ast.Typedef):
        print("Typedef")
    elif isinstance (node, c_ast.Typename):
        print("Typename")
        # Typename[name, quals, type*]
        
        '''
        Typename <params[0]>: name=None, quals=[]
            TypeDecl <type>: declname=None, quals=[]
              IdentifierType <type>: names=['void']
        '''
        
               
        name = node.name
        if name:
            if name in operands:
                operands[name] += 1
            else:
                operands[name] = 1
        
        # const, volatile
        quals = node.quals        
        if quals: # if true
            if (len(quals) == 2):
                print(quals[0]+"   "+quals[1])
                if (quals[0] in operators):
                    operators[quals[0]] += 1
                else:
                    operators[quals[0]] = 1
                if (quals[1] in operators):
                    operators[quals[1]] += 1
                else:
                    operators[quals[1]] = 1
            else: #(node.quals() == 1):
                #print("******************  "+node.quals[0])
                if (quals[0] in operators):
                    operators[quals[0]] += 1
                else:
                    operators[quals[0]] = 1
                    
        #print("**** "+str(type(node.type)))
        
              
        # !!! GRESKA
        if node.type:
        #    print("!!! "+str(type(node.type)))
            traverse(node.type)            
        
    elif isinstance (node, c_ast.UnaryOp):
        print("unary operator")
    elif isinstance (node, c_ast.Union):
        print("Union")  
    elif isinstance (node, c_ast.While):
        print("while")           
    else:
        print("Error!")
        return 
    
    return

In [90]:
# FileAst[ext**]
for node in ast.ext:
    print(node)
    traverse(node)

FuncDef
Decl: name=foo, quals=['const'], storage=[], funcspec=[]
  FuncDecl <type>: 
    ParamList <args>: 
      Typename <params[0]>: name=None, quals=[]
        TypeDecl <type>: declname=None, quals=[]
          IdentifierType <type>: names=['void']
    TypeDecl <type>: declname=foo, quals=['const']
      IdentifierType <type>: names=['char']
Decl
Func decl
Param list
Typename
Type Decl
Identifier Type
Type Decl
Identifier Type
Compound
{
Decl
Type Decl
Identifier Type
Constant
Return
ID
}
-----------------
FuncDef
Decl: name=maxout_in, quals=[], storage=[], funcspec=[]
  FuncDecl <type>: 
    ParamList <args>: 
      Decl <params[0]>: name=paste, quals=[], storage=[], funcspec=[]
        TypeDecl <type>: declname=paste, quals=[]
          IdentifierType <type>: names=['int']
      Decl <params[1]>: name=matrix, quals=[], storage=[], funcspec=[]
        PtrDecl <type>: quals=[]
          PtrDecl <type>: quals=[]
            TypeDecl <type>: declname=matrix, quals=[]
              Id

In [91]:
print(operands)
print(operators)

{'void ': 1, 'char ': 3, 'c': 1, 'int ': 5, 'char **': 1, 'matrix': 1, 'paste': 1, 'char *': 1, 'b': 1}
{';': 10, 'const': 2, '{': 3, '}': 3, 'return': 3, '-': 1, '*': 1, '(': 1, ')': 1, 'auto': 1, '=': 1}
